# Wine Quality Data ETL & Data Quality Analysis
Objective: Data Quality Analysis 
                                                                                            
Dataset: Wine Quality (Red and White) 
                                                                                            
Dimensions: Validity, Consistency, Completeness

In [91]:
#Intro and Imports to set up the two datasets.

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

red_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
white_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv"

red = pd.read_csv(red_url, sep=';')
white = pd.read_csv(white_url, sep=';')

red['wine_type'] = 'red'
white['wine_type'] = 'white'
print("Dataset shape (red):", red.shape)
print("Dataset shape (white):", white.shape)

display(red.head()), display(white.head())


Dataset shape (red): (1599, 13)
Dataset shape (white): (4898, 13)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wine_type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,red
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,red
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,red
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wine_type
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,white
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,white
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,white
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,white
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,white


(None, None)

In [92]:
# Completeness Check
missing_stats = pd.DataFrame({
    'Column': red.columns,
    'Missing_Count': red.isnull().sum(),
    'Missing_Percentage': (red.isnull().sum() / len(red)) * 100
})


print("Red wine shape:", red.shape)
print("White wine shape:", white.shape)

missing_stats = missing_stats[missing_stats['Missing_Count'] > 0].sort_values('Missing_Percentage', ascending=False)
if missing_stats.empty:
    display(HTML('<p style="color:red"><b>No missing Red Wine values found!</b></p>'))

print("\nMissing values (red):")
print(red.isnull().sum())

missing_stats = pd.DataFrame({
    'Column': white.columns,
    'Missing_Count': white.isnull().sum(),
    'Missing_Percentage': (white.isnull().sum() / len(red)) * 100
})

missing_stats = missing_stats[missing_stats['Missing_Count'] > 0].sort_values('Missing_Percentage', ascending=False)
if missing_stats.empty:
    display(HTML('<p style="color:red"><b>No missing White Wine values found!</b></p>'))

print("\nMissing values (White):")
print(white.isnull().sum())


Red wine shape: (1599, 13)
White wine shape: (4898, 13)



Missing values (red):
fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
wine_type               0
dtype: int64



Missing values (White):
fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
wine_type               0
dtype: int64


In [4]:
# Combine datasets
combined = pd.concat([red, white], ignore_index=True)
print("Combined shape:", combined.shape)
combined.head()

Combined shape: (6497, 13)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wine_type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,red
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,red
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,red
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red


In [94]:
# Validity: Negative or impossible values
# Define validity ranges for key numeric columns (adjust per dataset)
valid_ranges = {
    'fixed acidity': (3.8, 15.9),
    'volatile acidity': (0.008, 1.6),
    'citric acid': (0.0, 1.7),
    'residual sugar': (0.6, 50),
    'chlorides': (0.009, 0.611),
    'free sulfur dioxide': (1, 132),
    'total sulfur dioxide': (6, 440),
    'density': (0.9, 1.2),
    'pH': (1, 14),
    'sulphates': (0.22, 2.0),
    'alcohol': (0, 14),
    'quality': (0, 10),
}

invalid_rows = {}

for col in combined.columns:
    if pd.api.types.is_numeric_dtype(combined[col]):
        # Ensure column is numeric (in case it's read as string)
        combined[col] = pd.to_numeric(combined[col], errors='coerce')
        min_val, max_val = valid_ranges.get(col, (None, None))
        
        # Create mask for invalid values
        mask = combined[col].isna() | (combined[col] < min_val) | (combined[col] > max_val)

        
        ## Store invalid rows with wine_type
        if mask.any():
            invalid_rows[col] = combined.loc[mask, ['wine_type', col]]

output_folder = '..\output'            

# Display results
if not invalid_rows:
    display(HTML('<p style="color:red"><b>No invalid numeric values found!</b></p>'))
else:
    print("\nInvalid numeric values:")
    for col, df_invalid in invalid_rows.items():
        filename = f"{col.replace(' ', '_')}_invalid_values.csv"
        path = os.path.join(output_folder, filename)
        print(f"\nColumn: {col}")
        display(df_invalid)
        df_invalid.to_csv(path, index=False)




Invalid numeric values:

Column: residual sugar


,wine_type,residual sugar
4380,white,65.8



Column: free sulfur dioxide


,wine_type,free sulfur dioxide
3530,white,146.5
4649,white,138.5
6344,white,289.0



Column: alcohol


,wine_type,alcohol
652,red,14.90
5517,white,14.20
6102,white,14.05


In [80]:
# --- Consistency Check ---

# Duplicate row detection
duplicates = combined[combined.duplicated(subset=[col for col in combined.columns], keep=False)]
if duplicates.empty:
    display(HTML('<p style="color:green"><b>No duplicate rows found!</b></p>'))
else:
    print(f"Found {len(duplicates)} duplicate rows:")
    display(duplicates.head(20))  
    duplcicates.to_csv('../output/duplicates.csv', index=False)

#count duplicates per wine_type
if not duplicates.empty and 'wine_type' in duplicates.columns:
    dup_counts_by_type = duplicates['wine_type'].value_counts()
    dup_counts_clean = dup_counts_by_type.reset_index()
    dup_counts_clean.columns = ['wine_type', 'duplicate_count']
    display(dup_counts_clean)
    dup_counts_clean.to_csv('../output/duplicates_per_wine_type.csv', index=False)
    
combined_cleaned = combined.drop_duplicates(ignore_index=True)

print(f"Rows before removing duplicates: {len(combined)}")
print(f"Rows after removing duplicates: {len(combined_cleaned)}")

# Schema alignment
expected_schema = {
    'fixed acidity': 'float',
    'volatile acidity': 'float',
    'citric acid': 'float',
    'residual sugar': 'float',
    'chlorides': 'float',
    'free sulfur dioxide': 'float',
    'total sulfur dioxide': 'float',
    'density': 'float',
    'pH': 'float',
    'sulphates': 'float',
    'alcohol': 'float',
    'quality': 'int',
    'wine_type': 'object'
}

schema_issues = {}
for col, dtype in expected_schema.items():
    if col not in combined.columns:
        schema_issues[col] = 'Missing'
    else:
        # Use pandas type checking
        if np.issubdtype(combined[col].dtype, np.number):
            col_type = 'float' if combined[col].dtype.kind == 'f' else 'int'
        else:
            col_type = 'object'
        if col_type != dtype:
            schema_issues[col] = f"Expected {dtype}, got {col_type}"

if schema_issues:
    print("Schema issues detected:")
    for col, issue in schema_issues.items():
        print(f"- {col}: {issue}")
else:
    display(HTML('<p style="color:green"><b>\n\nAll columns match expected schema!</b></p>'))


Found 2169 duplicate rows:


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wine_type
0,7.4,0.700,0.00,1.90,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red
4,7.4,0.700,0.00,1.90,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red
9,7.5,0.500,0.36,6.10,0.071,17.0,102.0,0.9978,3.35,0.80,10.5,5,red
11,7.5,0.500,0.36,6.10,0.071,17.0,102.0,0.9978,3.35,0.80,10.5,5,red
22,7.9,0.430,0.21,1.60,0.106,10.0,37.0,0.9966,3.17,0.91,9.5,5,red
27,7.9,0.430,0.21,1.60,0.106,10.0,37.0,0.9966,3.17,0.91,9.5,5,red
39,7.3,0.450,0.36,5.90,0.074,12.0,87.0,0.9978,3.33,0.83,10.5,5,red
40,7.3,0.450,0.36,5.90,0.074,12.0,87.0,0.9978,3.33,0.83,10.5,5,red
64,7.2,0.725,0.05,4.65,0.086,4.0,11.0,0.9962,3.41,0.39,10.9,5,red
65,7.2,0.725,0.05,4.65,0.086,4.0,11.0,0.9962,3.41,0.39,10.9,5,red


,wine_type,duplicate_count
0,white,1709
1,red,460


Rows before removing duplicates: 6497
Rows after removing duplicates: 5320


In [5]:
# Check schema consistency
print("Red columns == White columns:", list(red.columns) == list(white.columns))

# Compare distributions across wine types
combined.groupby('wine_type').mean()

Red columns == White columns: True


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
wine_type,,,,,,,,,,,,
red,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
white,6.854788,0.278241,0.334192,6.391415,0.045772,35.308085,138.360657,0.994027,3.188267,0.489847,10.514267,5.877909


In [84]:
#Save cleaned dataset
combined_cleaned.to_csv('../output/combined_cleaned_dataset.csv', index=False)
print("Cleaned dataset saved successfully!")

Cleaned dataset saved successfully!
